## Запуск Spark

In [ ]:
import sys
import os

spark_home='/opt/spark-1.5.1-bin-hadoop2.6/'
# addin pyspark to current path
sys.path.append( spark_home+'/python' )
sys.path.append( spark_home+'/python/lib/py4j-0.8.2.1-src.zip' )

# Установка локальных переменных
os.environ["SPARK_HOME"] = spark_home
os.environ["HADOOP_HOME"] = '/opt/cloudera/parcels/CDH/lib/hadoop'
os.environ["HADOOP_YARN_HOME"] = '/opt/cloudera/parcels/CDH/lib/hadoop-yarn'
os.environ["YARN_CONF_DIR"] = '/etc/hadoop/conf.cloudera.yarn'
os.environ["SPARK_CLASSPATH"] = '/etc/hive/conf.cloudera.hive1'
os.environ["PYSPARK_SUBMIT_ARGS"] = '--master local[4] pyspark-shell'

from pyspark import SparkContext, SparkConf
from pyspark.sql import HiveContext
conf = SparkConf ().set( 'spark.app.name', 'test');
sc = SparkContext (conf= conf)

Загрузка датасета, с которым мы проводим эсперименты

In [ ]:
# загружаем датасет
from pyspark.mllib.util import MLUtils
examples = MLUtils.loadLibSVMFile(sc, "/testdata/mllib_demo/libsvm_dataset")


# делим выборку на обучающую и тестовую
(train, test) = examples.randomSplit ([0.7, 0.3], seed=1)

# <br><br><br><br>Линейные классификаторы

<img src="images/linear_classifier.png" width=300>

Обучение лин. классификатора - задача выпуклой минимизации функционала от вектора весов $w$

$$f(w) = \lambda R(w) + \frac{1}{n} \sum_{i=1}^n L(w;x_i,y_i) $$

где $x_i \in  \mathbb{R}$ - обучающие примеры,<br><br>
$y \in \{-1,+1\}$ - соответствующие метки<br><br>
$L(w;x,y)$ - функция потерь<br><br>
$R(w)$ - функция регуляризации

<br>**Функции потерь** имеют вид:

- Для SVM: $L(w;x,y) = \max\{0,1-yw^Tx\}, y \in \{-1, +1\}$<br><br>
- Для логистической регрессии: $L(w;x,y) = log(1+\exp(-yw^Tx)), y \in \{-1,+1\}$<br><br>

**Функции регуляризации** могут быть:

- L1 - $||w||_2$ - дает более разреженные и интерпретируемые результаты
- L2 - $\frac{1}{2}||w||_2^2$ - более простая

Все эти задачи решаются в MLlib метдом стохастического градиентного спуска SGD

- SVM: cl.SVMWithSGD
- Логистическая регрессия: cl.LogisticRegressionWithSGD

classifier.train(trainDataset) - обучает модель
Параметры:
- **data** – обучающая выборка RDD of LabeledPoint.
- **iterations** – число итераций (по умолчанию: 100).
- **step** – шаг SGD (по умолчанию: 1.0).
- **miniBatchFraction** – доля данных которая используется на каждой SGD итерации
- **initialWeights** – начальные веса (по умолчанию None)
- **regParam** – параметр регуляризации (по умолчанию: 0.01)
- **regType** – Тип регуляризатора (по умолчанию "l2")
 - **"l1"** для l1 регуляризации
 - **"l2"** для l2 регуляризации
 - **None** для отсутствия регуляризации
- **intercept** – Булевый параметр, показывающий, применять ли дополнительное представление обучающих данных (то есть включены ли смещенные фичи или нет) ???


In [ ]:
import pyspark.mllib.classification as cl

model = cl.LogisticRegressionWithSGD.train (train, 
    iterations=500, 
    regType="l1")

Классифицируем результаты

In [ ]:
def plotROC (real_labels, predicted_labels):
    from sklearn.metrics import roc_curve, auc
    fpr, tpr, _ = roc_curve(real_labels, predicted_labels)
    roc_auc = auc (fpr, tpr)

    %matplotlib inline
    import matplotlib.pyplot as plt
    plt.clf()
    plt.plot(fpr,tpr)
    print "ROC AUC =", roc_auc

Чтобы модель выдавала непрерывный результат, надо удалить у нее пороговое значение model.clearThreshold()

In [ ]:
model.clearThreshold()
plotROC (test.map (lambda x: x.label).collect(), 
         test.map (lambda x:model.predict(x.features)).collect())

# <br><br><br><br>Линейная регрессия 

Мы решает ту же задачу
$$f(w) = \lambda R(w) + \frac{1}{n} \sum_{i=1}^n L(w;x_i,y_i) $$

Но функциноал потерь имеет вид <br> $L(w;x,y) = \frac{1}{2} (w^Tx-y)^2, y \in \mathbb{R}$<br>

Разные типы регуляризации задают разные методы:

- Если $R(w)=0$ то это **линейная регрессия** LinearRegressionWithSGD
- Если $R(w)=||w||_2$ - **лассо** LassoWithSGD
- Если $R(w)=\frac{1}{2}||w||_2^2$ - **регрессия Риджа** RidgeRegressionWithSGD


Параметры обучения модели такие же как и у линейных классификаторов:<br>
regression.train параметры:<br>
- **data** – обучающая выборка RDD of LabeledPoint.
- **iterations** – число итераций (по умолчанию: 100).
- **step** – шаг SGD (по умолчанию: 1.0).
- **miniBatchFraction** – доля данных которая используется на каждой SGD итерации
- **initialWeights** – начальные веса (по умолчанию None)
- **regParam** – параметр регуляризации (по умолчанию: 0.01)
- **regType** – Тип регуляризатора (по умолчанию None)
 - **"l1"** для l1 регуляризации (лассо)
 - **"l2"** для l2 регуляризации (ридж)
 - **None** для отсутствия регуляризации
- **intercept** – Булевый параметр, показывающий, применять ли дополнительное представление обучающих данных (то есть включены ли смещенные фичи или нет) ???


In [ ]:
import pyspark.mllib.regression as reg
model = reg.RidgeRegressionWithSGD.train (train)

In [ ]:
plotROC (test.map(lambda x:x.label).collect(),
         test.map(lambda x: model.predict(x.features)).collect())

# <br><br><br><br>Наивный байес

<img src='images/Bayes.png'>

Алгоритм обучения наивного байеса NaiveBayes.train принимает на вход всего 2 параметра
- **data** обучающую выборку
- **lab** параметр сглаживания

In [ ]:
?NaiveBayes.train(train, 10.0)

In [ ]:
from pyspark.mllib.classification import NaiveBayes

model = NaiveBayes.train(train, 10.0)

In [ ]:
model.predict(test.take(10)[5].features)

In [ ]:
plotROC (test.map (lambda x: x.label).collect(),
         test.map (lambda x: model.predict (x.features)).collect())

# <br><br><br><br>Решающие деревья

<img src="images/decision_tree_example.png" width=400>

При обучении дерева максимизируеся impuruty после разбиения

- gini:  $\sum_{i=1}^{C} f_i (1-f_i)$<br><br>
- entropy $\sum_{i=1}^{C} -f_i log(f_i)$<br><br>
- vairance $\frac{1}{N} \sum_{i=1}^{N} (y_i-\mu)^2$

Обучение алгоритматами
DecisionTree.trainClassifier
и
DecisionTree.trainRegressor

с параметрами 
- **data** – обучающая выборка, RDD
- **numClasses** – число классов (только для классификатора)
- **categoricalFeaturesInfo** – указание, какие у категориальных фич есть категории
- **impurity** – Возможные значения
 - **“entropy”** (для классификатора)
 - **“gini”** (для классификатора)
 - **“variance”** (для регрессии)
- **maxDepth** – макс глубина дерева. Глубина 0 - 1 лист, Глубина 1 - 1 внутренняя нода и 2 листа
- **maxBins** – число бакетов, по которому ищутся сплиты по каждому из  узлов.
- **minInstancesPerNode** – мин. число инстансов, которые требуются дочерним нодам для созданя родительсткого сплита
- **minInfoGain** – минимальный уровень информации, для создания сплита


In [ ]:
from pyspark.mllib.tree import DecisionTree

model = DecisionTree.trainClassifier(train, 
                                     numClasses=2, 
                                     categoricalFeaturesInfo={},
                                     impurity='entropy',
                                     maxDepth=3, 
                                     maxBins=16)

In [ ]:
plotROC (test.map (lambda x: x.label).collect(),
         model.predict(test.map (lambda x: x.features)).collect())

In [ ]:
print model.toDebugString()

# <br><br><br><br>Случайный лес

<img src="images/random_forest.png" width=300>

random forest использует деревья, описанные выше.
Так же как и у деревьев, у случайного леса есть методы
- **trainClassifier** для обучения классификатора
- **trainRegression** для обучения функции регрессии

Параметры:

- **data** – обучающая выборка, RDD LabeledPoint. Классов может быть несколько
- **numClasses** – число классов для классификации
- **categoricalFeaturesInfo** – информация по категорийным фичам. Запись (n -> k) показывает, что фича n - категориальная и имеет k категорий, начинащихся с 0: $\{0, 1, ..., k-1\}$.
- **numTrees** – число деревьев в лесу
- **featureSubsetStrategy** - число фич, которые рассматриваются для сплитов на каждой из нод. Возможные варианты:
 -**“auto”** (по умолчаниюб если дерево 1- выбираются все фичи, иначе берется корень из числа фич)
 -**“all”**
 -**“sqrt”**
 -**“log2”**
 -**“onethird”**
- **impurity** – критерий вычисления information gain. Значения:
 - “gini” (для классификации, рекомендуются)
 - “entropy” (для классификации)
 - "variance" (для задач регрессии)
- **maxDepth** – максимальная глубина дерева
- **maxBins** – максимальное число корзин, используемое для разделения фич (по умолчанию 100)
- **seed** – Случайный seed для бутстрепинга и выбора подмножеств фич

In [ ]:
?RandomForest.trainClassifier()

In [ ]:
from pyspark.mllib.tree import RandomForest
model = RandomForest.trainClassifier(train, 
                                     numClasses=2, 
                                     categoricalFeaturesInfo={},
                                     numTrees=300, 
                                     featureSubsetStrategy="auto",
                                     impurity='gini', 
                                     maxDepth=20, 
                                     maxBins=32,
                                     seed=13)

In [ ]:
plotROC (test.map(lambda x: x.label).collect(),
         model.predict (test.map(lambda x:x.features)).collect())